In [10]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal



import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [11]:
from common import EEG_fif
from common import create_dataloader

In [12]:
torch.cuda.is_available()

True

In [ ]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = ''
base_url = 'https://physionet.org/files/eegmmidb/'
#runs = [4, 6, 8, 10, 12,14]
#runs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#runs = [3,4,5,6,7,8,9,10,11,12,13,14]

#runs = [11,13,15,17]

#runs = [4,6,8,10]
runs = [3,5,7,9]


#runs = [3,5,7,9,4,6,8,10]



#runs = [3]
subjects = [20,31,48,50,54,55,59,62]

print(path)
eeg = EEG_fif(path, base_url, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter
#freq = (1., 45.)

#raw=eeg.raw_ica(n_components=8,max_iter=1000)
raw=raw.notch_filter([50,75,100])
raw=raw.filter( 8,14, method='fir', verbose=20)


print("Filter done")

In [ ]:
raw.ch_names

In [ ]:
#raw = raw.drop_channels(['C3'])
raw=raw.pick_channels(['C3', 'C4','STIM MARKERS'])
raw.ch_names

In [ ]:
#epochs=eeg.epochs_visu(raw,tmin=0,tmax=6,baseline=(0,3))
epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,2))

In [ ]:
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)
#X=X[:,:,int(0.25*250):int(1750-(250*1))]
#X = X[:, np.newaxis,:,:]
print(X.shape)
y=y-1
y


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

print('train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

In [ ]:

batch_size = X_train.shape[2]

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

in_channels  = X.shape[1]  #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5


In [ ]:
X_train = torch.FloatTensor(X_train)
fake_eeg_data = torch.randn_like(X_train) #(batch_size, channel, length)

In [ ]:
        
        #using sequential helps bind multiple operations together
layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU()
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        )
        

In [ ]:
out = layer1(fake_eeg_data)
out.shape

In [ ]:
out = layer2(out)
out.shape

In [ ]:
out = layer3(out)
out.shape

In [ ]:
out = out.reshape(out.size(0), -1)

linear_shape = out.shape[1]
linear_shape

In [ ]:
fc = nn.Linear(linear_shape, out_size)
out = fc(out)
out.shape

In [ ]:


# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, out_size):
        super().__init__()
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        
        
        self.fc = nn.Linear(linear_shape  , out_size)
        #self.fc = nn.Linear(80064 , 2)
        
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = self.layer3(out)
        #out = self.drop_out(out)
        #out = self.layer4(out)
        #out = self.drop_out(out)

        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

### Freeze layer

In [ ]:
#del net
net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)

In [ ]:
net.load_state_dict(torch.load('CNN_3layer_2class_S54_EX/0.6529_CNN_3layer_2class_S54_EX_0.6529_69.5652'))

In [ ]:
parms = net.state_dict()
parms.keys()

In [ ]:
for name,param in net.named_parameters():
    if param.requires_grad and 'layer1' in name:
        param.requires_grad = False
    if param.requires_grad and 'layer2' in name:
        param.requires_grad = False
    # if param.requires_grad and 'layer3' in name:
    #     param.requires_grad = False

In [ ]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"OurData_CNN_3layer_cuttime_2class_S54_findtune_freeze_1_2_online_EX", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.00000001,
      "architecture": "CNN",
      "dataset": "S20",
      "epochs": 1000000,
      "weightname":"OurData_CNN_3layer_cuttime_2class_S54_findtune_freeze_1_2_online_EX",
      "num_step_per_epoch" : num_step, 
        
      }
    )




config = wand.config
print(config.num_step_per_epoch)


In [ ]:
from common import train
#net = ConvNet().cuda(0)
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()


train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    gpu_num = 0,
    train_loader = train_loader,
    test_loader = test_loader,
    optimizer = optimizer  ,
    criterion = criterion ,
    vail_loader = None,
    wand = wand
         )


wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

### Validate Findtune

In [ ]:
del net

In [ ]:
net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)

In [ ]:
net.load_state_dict(torch.load('OurData_CNN_3layer_cuttime_2class_S49_findtune_freeze_1_2_online_ex_find_MI_C3_C4/0.5584_OurData_CNN_3layer_cuttime_2class_S49_findtune_freeze_1_2_online_ex_find_MI_C3_C4_0.5584_83.3333'))

In [ ]:
# home directory + datasets folder
#path = '/content/drive/MyDrive/MNE-eegbci-data/files/eegmmidb/'
path = ''
base_url = 'https://physionet.org/files/eegmmidb/'
#runs = [4, 6, 8, 10, 12,14]
#runs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#runs = [3,4,5,6,7,8,9,10,11,12,13,14]
runs = [4]
#runs = [11,13,15,17]
#runs = [4,6,8,10]
#runs = [3,5,7,9,4,6,8,10]
subjects = [1]

print(path)
eeg = EEG_fif(path, base_url, subjects, runs)
raw=eeg.data_to_raw()
print(raw)
print("Raw done")
# apply filter
#freq = (1., 45.)

#raw=eeg.raw_ica(n_components=8,max_iter=1000)
raw=raw.notch_filter([50,75,100])
raw=raw.filter( 8,14, method='fir', verbose=20)


print("Filter done")

In [ ]:
raw.ch_names
raw = raw.drop_channels(['Fp1', 'Fp2', 'P7', 'P8', 'O1', 'O2'])
raw.ch_names

In [ ]:
#epochs=eeg.epochs_visu(raw,tmin=0,tmax=6,baseline=(0,3))
epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,2))

In [ ]:
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)

#X=X[:,:,int(0.25*250):1500]
X=X[:,:,int(0.25*250):int(1750-(250*1))]
#X = X[:, np.newaxis,:,:]
print(X.shape)
y=y-1
y

In [ ]:

batch_size = 1438





In [ ]:
test_loader = create_dataloader(X, y, batch_size=batch_size)
num_step =math.ceil(len(test_loader.dataset) / batch_size)

In [ ]:
# Train the model
num_epochs = 1

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []
old_acc = 0
for epoch in range(num_epochs):

    loss = 0.0
    correct = 0
    iterations = 0
    net.eval()
    for i, (items, classes) in enumerate(test_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(0)
            classes = classes.cuda(0)
        
        outputs = net(items)
        #loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        
        for i in range(predicted.shape[0]):
            
            if predicted[i] == classes.data[i]:
                correct += 1
            
        #correct += (predicted == classes.data).sum()
        
        iterations += 1

    #valid_loss.append(loss/iterations)
    #correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct / len(test_loader.dataset) * 100.0)
print (' Val Acc: %.4f'
                   %(valid_accuracy[-1]))


In [ ]:
predicted


In [ ]:
y_true=classes.data.cpu().numpy()
y_true

In [ ]:
predicted

In [ ]:
y_pred=predicted.cpu().numpy()
y_pred

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_true, y_pred)

### Validate Findtune V2

In [ ]:
del net

In [ ]:
net = ConvNet().cuda(0)

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
net.load_state_dict(torch.load('OurData_CNN_4layer_2ch_2class_S20_EX_10_Nov_findtune/0.5425_OurData_CNN_3layer_cuttime_2class_S20_EX_findtune_freeze_1_2_online_MI_C3_C4_0.5425_88.8889'))

In [ ]:
# Train the model
num_epochs = 1

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []
old_acc = 0
for epoch in range(num_epochs):

    loss = 0.0
    correct = 0
    iterations = 0
    net.eval()
    for i, (items, classes) in enumerate(test_loader):
        items = Variable(items)
        classes = Variable(classes)
        
        if cuda.is_available():
            items = items.cuda(1)
            classes = classes.cuda(1)
        
        outputs = net(items)
        loss += criterion(outputs, classes).item()
        
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == classes.data).sum()
        
        iterations += 1

    valid_loss.append(loss/iterations)
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(test_loader.dataset) * 100.0)
print (' Val Acc: %.4f'
                   %(valid_accuracy[-1]))


In [ ]:
y

In [ ]:
y_pred=predicted.cpu().numpy()


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
confusion_matrix(y, y_pred)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y, y_pred)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
from common import train
from common import create_dataloader
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import wandb
wandb.login()
os.environ['http_proxy'] = "http://192.41.170.23:3128" 
os.environ['https_proxy'] = "http://192.41.170.23:3128" 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nutapol-1997 (eeg_mi). Use `wandb login --relogin` to force relogin


In [2]:
from common import EEG_fif
from common import create_dataloader

In [3]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, out_size):
        super().__init__()
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        
        
        self.fc = nn.Linear(linear_shape  , out_size)
        #self.fc = nn.Linear(80064 , 2)
        
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = self.layer3(out)
        #out = self.drop_out(out)
        #out = self.layer4(out)
        #out = self.drop_out(out)

        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [4]:
subjects =  [31, 48, 50, 54, 59, 62] 
print("subjects number : {} ".format(subjects))

#[20,31,47,48,49,54,55,59,62]# list(range(36, 63))

in_channels  = 2 #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5



path = ''
base_url = 'https://physionet.org/files/eegmmidb/'






subjects number : [31, 48, 50, 54, 59, 62] 


In [5]:
for subject in subjects:
    for i in range(3):
        if i == 0:
            n_type = "EX"
            runs = [3,5,7,9]
        if i == 1:
            n_type = "MI"
            runs = [4,6,8,10]
        if i == 2:
            n_type = "ALL"
            runs = [3,5,7,9,4,6,8,10]

        path_f_weight = "CNN_3layer_2class_S20_{0}_X/".format(n_type)
        dir_list = os.listdir(path_f_weight)
        dir_sort = np.asarray(dir_list).sort()
        part_weight = "CNN_3layer_2class_S20_{0}_X/{1}".format(n_type,dir_list[-1]) 
        
        print("############## {} ########".format(subject))
        
        eeg = EEG_fif(path, base_url, subjects, runs)
        raw=eeg.data_to_raw()
        print("Raw done")
        raw=raw.notch_filter([50])
        raw=raw.filter( 8,14, method='fir', verbose=20)
        #raw.resample(160)
        print("Filter done")
        raw.pick_channels(['C3', 'C4', 'STIM MARKERS'])
        epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,1))
        X, y = eeg.get_X_y(epochs)
        #X=X[:,:,int(0.25*250):int(1750-(250*1))]
        #X=X[:,:,int(2*160):int(1211-(250*1))]
        y=y-1
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)


        print('train size',X_train.shape, y_train.shape)
        print('Test size',X_test.shape, y_test.shape)
        batch_size = X_train.shape[2]
        train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
        test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
        num_step =math.ceil(len(train_loader.dataset) / batch_size)

        x_train = torch.FloatTensor(X_train)
        fake_eeg_data = torch.randn_like(x_train) #(batch_size, channel, length)

            #using sequential helps bind multiple operations together
        layer1 = nn.Sequential(
                    #padding = (kernel_size - 1) / 2 = 2
                    nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer2 = nn.Sequential(
                    nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels*2),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer3 = nn.Sequential(
                    nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                )


        out = layer1(fake_eeg_data)
        out = layer2(out)
        out = layer3(out)
        out = out.reshape(out.size(0), -1)
        linear_shape = out.shape[1]
        fc = nn.Linear(linear_shape, out_size)
        out = fc(out)

        wand = wandb.init(

              # Set the project where this run will be logged
              project="Motor-Imagery", 
              # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
              name="CNN_3layer_2class_sunsun_S{0}_{1}".format(subject,n_type), 
              # Track hyperparameters and run metadata
              config={
              "learning_rate": 0.00001,
              "architecture": "CNN",
              "dataset": "S{0}".format(subject),
              "epochs": 1000,
              "weightname":"CNN_3layer_2class_sunsun_S{0}_{1}".format(subject,n_type),
              "num_step_per_epoch" : num_step, 

              }
            )
        wandb.login()
        config = wand.config
        net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
        net.load_state_dict(torch.load(part_weight))
        for name,param in net.named_parameters():
            if param.requires_grad and 'layer1' in name:
                param.requires_grad = False
            if param.requires_grad and 'layer2' in name:
                param.requires_grad = False
        
        optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
        criterion = nn.CrossEntropyLoss()


        train_loss,valid_loss,train_accuracy,valid_accuracy =train(
            model = net,
            gpu_num = 0,
            train_loader = train_loader,
            test_loader = test_loader,
            optimizer = optimizer  ,
            criterion = criterion ,
            wand = wand
                 )


        wandb.alert(
                    title='Finish',
                    text=f'Finishing training',
                )

############## 31 ########



/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S031/S031R03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S031/S031R05.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S031/S031R07.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fi

Raw done
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 sec)

Filtering raw data in 24 contiguous segments
Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished


Filter done
1440 events found


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/nutapolt/eeg_mi/common.py:657: RuntimeWarning: No matching events found for 3 (event id 3)
  epochs = mne.Epochs(


Not setting metadata
960 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 960 events and 1751 original time points ...
48 bad epochs dropped
train size (638, 2, 1751) (638,)
Test size (274, 2, 1751) (274,)


Test/Test_accuracy,▁▁▁▁▄██▇▇▇▇▆▅▅▅▅▆▆▆▆▆▅▅▅▅▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅
Test/Test_loss,▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_accuracy,▁▃▅▅▆▆▇▇▇▇██████████████████████████████
train/train_loss,█▇▆▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test/Test_accuracy,59.85401
Test/Test_loss,1.09911
train/train_accuracy,100.0
train/train_loss,0.02153


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch 1/1000, Tr Loss: 1.3267, Tr Acc: 56.4263, Val Loss: 1.3020, Val Acc: 55.8394
Epoch 101/1000, Tr Loss: 0.4493, Tr Acc: 80.2508, Val Loss: 0.7346, Val Acc: 61.3139
Epoch 201/1000, Tr Loss: 0.2860, Tr Acc: 90.2821, Val Loss: 0.7379, Val Acc: 60.9489
Epoch 301/1000, Tr Loss: 0.2116, Tr Acc: 95.7680, Val Loss: 0.7869, Val Acc: 64.2336
Epoch 401/1000, Tr Loss: 0.1598, Tr Acc: 97.3354, Val Loss: 0.8573, Val Acc: 64.2336
Epoch 501/1000, Tr Loss: 0.1186, Tr Acc: 98.5893, Val Loss: 0.9362, Val Acc: 63.1387
Epoch 601/1000, Tr Loss: 0.0914, Tr Acc: 99.5298, Val Loss: 1.0067, Val Acc: 62.0438
Epoch 701/1000, Tr Loss: 0.0787, Tr Acc: 99.5298, Val Loss: 1.0787, Val Acc: 59.8540
Epoch 801/1000, Tr Loss: 0.0626, Tr Acc: 99.8433, Val Loss: 1.1462, Val Acc: 60.2190
Epoch 901/1000, Tr Loss: 0.0522, Tr Acc: 99.8433, Val Loss: 1.2057, Val Acc: 60.2190


5